## Step 7 - Prepare the application for the Production Centre

The Production Centre uses Docker images and deploys them in a Cloud based environment for large scale production campaigns.

This step will trigger the build of a Docker image containing the data transformation application created in the previous steps 

We will invoke components of the Ellip Platform Continuous Integration.

The input for trigering the build of Docker image containing the data transformation application uses the pom, an XML file that contains information about the data transformation application and configuration details used by Maven to build the Docker image.

The goal of step 7 is to create a Jupyter Notebook that will read the data transformation application pom file and trigger the build of the Docker image.

#### Reading the Pom

Create a simple Python class to read the elements of the pom.

* First the import of the required Python libraries:

In [2]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import getpass 

* The class with a constructor taking as parameter the path to the pom file and methods to retrieve the version, artifact id and community. This information is used to create the build request



In [3]:
class Pom:

    root = None
    tree = None
    pom_namespaces = { 'A':'http://maven.apache.org/POM/4.0.0' }
    
    def __init__(self, pom_path):
        
        try:
            self.tree = etree.parse(pom_path)
            self.root = self.tree.getroot()

        except IOError as e:
            print 'Error, run this utility as you would run maven'
            
    def get_version(self):
    
        el = self.root.xpath('/A:project/A:version', 
                             namespaces=self.pom_namespaces)
        
        return el[0].text
    
    def get_artifact_id(self):
    
        el = self.root.xpath('/A:project/A:artifactId', 
                             namespaces=self.pom_namespaces)
        
        return el[0].text
    
    def get_community(self):
    
        el = self.root.xpath('/A:project/A:properties/A:community', 
                             namespaces=self.pom_namespaces)
        
        return el[0].text

* Create two simple functions to submit the build request and to monitor it:

In [4]:
def submit_build_job(server, job, sleepSecs=3):
    
    last_build = server.get_job_info(job)['lastCompletedBuild']['number']
    
    server.build_job(job, {'RELEASE': 'true'})
    
    while server.get_job_info(job)['lastBuild']['number'] == last_build:
        
        time.sleep(sleepSecs)
        
    return server.get_job_info(job)['lastBuild']['number']

In [5]:
def monitor_build_job(server, job, sleepSecs=15):
    
    last_build_number = server.get_job_info(job)['lastBuild']['number']
    
    while server.get_build_info(job, last_build_number)['building']:
        
        time.sleep(sleepSecs)
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'SUCCESS':
        
        return True
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'FAILURE':
        
        return False

* Set the authentication information:

In [6]:
username = raw_input("What is your Jenkins username? ")
api_key = getpass.getpass('And your Jenkins API key:')

What is your Jenkins username? fbrito
And your Jenkins API key:········


* Establish a connection with the build component of the Ellip Platform Continuous Integration:

In [7]:
server = jenkins.Jenkins(url='https://build.terradue.com',
                        username=username,
                        password=api_key)

* Create a pom object:

In [8]:
pom = Pom(pom_path='/workspace/training/wfp-01-01-01/pom.xml')

* Create the build request path:

In [10]:
repositories = ['Gitlab Groups',
                'Github Organizations']

job = 'communities/%s/%s/%s/docker' % (repositories[0], 
                                           pom.get_community(), 
                                           pom.get_artifact_id())

In [11]:
print(job)

communities/Gitlab Groups/ec-better/wfp-01-01-01/docker


* Submit the build job:

In [12]:
submit_build_job(server, job)

4

* Monitor the build request:

In [14]:
if monitor_build_job(server, job):
    print 'Job build completed'
else:
    print 'Job build failed'

Job build failed
